## CNN logistic


Only photos with max 960 in axis X and Y

MSE 



In [1]:
# !pip install keras

In [10]:
import sys
sys.path.append('../src')

import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from tensorflow.keras import Sequential, Input, layers
from keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.metrics import Precision, Recall

from utils.data import get_best_slice, select_tomo_ids

In [11]:
path_train_csv= '/Users/wm/code/mc-will/bacterial_motor/BYU_Locating-Bacterial_Flagellar_Motors_2025/data/train_labels.csv'
path_image= '/Users/wm/code/mc-will/bacterial_motor/BYU_Locating-Bacterial_Flagellar_Motors_2025/data/pictures_raw/train'

df = pd.read_csv (path_train_csv).copy()

# Sorting only motors 1 or 0
df= df[df['Number of motors']<=1]

# df.head(3)
# Describe the CSV
# col = [i.replace(' ','_').replace('(','').replace(')','') for i in df.columns]

# df[col]= df
# df= df.drop(columns = ['Motor axis 0','Motor axis 1','Motor axis 2','Array shape (axis 0)','Array shape (axis 1)','Array shape (axis 2)','Voxel spacing','Number of motors'])
# df


In [12]:
# id_list= select_tomo_ids(df, )
# id_list
# # best= get_best_slice(df, id_list)



In [13]:
"""
Function look the foto 72 in each 'tomo'
* Img size is too heavy at 960*960 pixel, so se test on 256*256

"""
def load_from_each_tomo(parent_folder,
                        img_size=(960, 960),
                        # img_size=(256,256),
                        # img_size=(128, 128),
                        ):

    slices = []
    # select the column of 'tomo_id' with only 1/0 Motors
    tomo_ids = df['tomo_id']
    # print (tomo_ids)

    # Loop to look each foto per Tomo_id
    for id in tomo_ids:
        # make the path btw parent_path and tomo_id
        tomo_path = os.path.join(parent_folder, id)
        # print(tomo_path)

        # Check if it is a real directory
        if os.path.isdir(tomo_path):
            # make a liste of all the photo per tomo
            all_slices = sorted([f for f in os.listdir(tomo_path)])
            # print(' all slices:',len(all_slices), type(all_slices), all_slices)

            number = 72 ### mean where the best photo is

            slice_path =os.path.join(tomo_path, all_slices[number])
            # load the img from slice_path with same size , in 1 chanel = W/B
            img = load_img(slice_path,
                           target_size=img_size,
                           color_mode='grayscale',
                           )
            # Normalization btw 0/1 divide by 255
            img_array = img_to_array(img) / 255

            # append to the slice list
            slices.append(img_array)

    # return un array to be used
    return np.array(slices)



In [14]:
# selection of y= 1 or 0 Motors

X =load_from_each_tomo(path_image, )
y= df['Number of motors']

In [7]:
def init_model_cnnlog(X):
    model = Sequential()
    model.add(Input(shape= X.shape[1:]))

    model.add(layers.Conv2D(32, (3, 3),
                            padding='same',
                            activation="relu"))
    model.add(layers.MaxPool2D(pool_size=(2, 2)))
    #
    model.add(layers.Conv2D(64, (3, 3),
                            padding='same',
                            activation="relu"))
    model.add(layers.MaxPool2D(pool_size=(2, 2)))

    model.add(layers.Flatten())

    model.add(layers.Dense(256,activation= 'relu'))

    model.add(layers.Dense(1, activation='sigmoid'))
    
    
    ########
    
    model.compile(loss= 'binary_crossentropy',
                  optimizer = 'adam',
                #   metrics=['mse', 'F1Score'], 
                  metrics=['accuracy',
                        #    'recall', 
                           'precision', 
                           'F1Score'
                           ],)
    
    return model


In [8]:
model = init_model_cnnlog(X)

In [9]:
def model_train (model, X_train_processed, y_train, batch_size=16 ,epochs=50):
    verbose=1
    validation_split=0.1
    es = EarlyStopping(patience=5,
                       restore_best_weights=True)

    history = model.fit(X_train_processed,
                        y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_split=validation_split,
                        callbacks=[es],
                        verbose=verbose)
    return history
history = model_train(model, X,y)
history

Epoch 1/50


2025-06-04 13:38:02.668747: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


ValueError: in user code:

    File "/Users/wm/.pyenv/versions/bacterial_motor/lib/python3.10/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/Users/wm/.pyenv/versions/bacterial_motor/lib/python3.10/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/wm/.pyenv/versions/bacterial_motor/lib/python3.10/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/Users/wm/.pyenv/versions/bacterial_motor/lib/python3.10/site-packages/keras/engine/training.py", line 998, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/Users/wm/.pyenv/versions/bacterial_motor/lib/python3.10/site-packages/keras/engine/training.py", line 1092, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weig

In [10]:
# history.history['']


In [ ]:
# plt.plot(history.history['mse'],c='b');
# plt.plot(history.history['loss'], c='r');
# plt.plot(history.history['val_loss'],c='g');
plt.plot(history.history['F1Score'], c='r');
# plt.plot(history.history['accuracy'],c='black');
# plt.plot(history.history['val_accuracy'],'b');
plt.plot(history.history['recall'], c= 'r');
# plt.plot(history.history['val_recall']);
plt.plot(history.history['precision'], c='g');
# plt.plot(history.history['val_precision'], c='b');


In [ ]:
model.summary()